In [1]:
import pandas as pd
import numpy as np
import os
import glob
from psychopy import visual, tools

os.chdir('../')
os.getcwd()

'/Users/sumiyaabdirashid/Desktop/PhD/code/git/PRF_Experiment_Checkers/Experiment'

# Analysis from pRF+AttnStim

In [2]:
f_names = glob.glob("logs/*/*.tsv")

all_logs = pd.DataFrame([])

for f in f_names:  
    df = pd.read_table(f, keep_default_na=True)

    # change NaNs to 0 
    df['duration'] = df['duration'].fillna(0)
    df['nr_frames'] = df['nr_frames'].fillna(0)

    # add columns for end times
#     df['end'] = df.onset + df.duration
#     df['end_abs'] = df.onset_abs + df.duration

    # add column for task and color
    df['task'] = f.split('/')[1].split('_')[-1]
    df['color'] = f.split('/')[1].split('_')[-2]
    df['run'] = f.split('/')[1].split('_')[3]
    
    all_logs = all_logs.append(df, ignore_index=True)

In [5]:
## run some tests

# does run exist? 
# does run contain responses?
# does run log color_balance and fix_intensity values correctly?

# all_logs[(all_logs.run == '5') & (all_logs.color_balance != 0.5)]
all_logs[(all_logs.run == '5')].head(25)

,trial_nr,onset,event_type,phase,response,nr_frames,color_balance,fix_intensity,onset_abs,duration,end,end_abs,task,color,run
2772,0,0.035229,stim,0,NaN,41.0,0.50,0.000,5.593706,0.341689,0.376918,5.935395,largeAF,color,5
2773,0,0.376918,stim,1,NaN,44.0,0.50,0.000,5.935395,0.374997,0.751916,6.310392,largeAF,color,5
2774,0,0.751916,stim,2,NaN,45.0,0.50,0.000,6.310392,0.374963,1.126879,6.685356,largeAF,color,5
2775,0,1.126879,stim,3,NaN,43.0,0.50,0.000,6.685356,0.375039,1.501918,7.060395,largeAF,color,5
2776,1,1.501918,stim,0,NaN,45.0,0.50,0.000,7.060395,0.375001,1.876919,7.435396,largeAF,color,5
2777,1,1.876919,stim,1,NaN,44.0,0.50,0.000,7.435396,0.374952,2.251871,7.810347,largeAF,color,5
2778,1,2.251871,stim,2,NaN,45.0,0.30,0.000,7.810347,0.374999,2.626870,8.185346,largeAF,color,5
2779,1,2.626870,stim,3,NaN,44.0,0.50,0.050,8.185346,0.375044,3.001914,8.560391,largeAF,color,5
2780,2,3.001914,stim,0,NaN,45.0,0.30,0.000,8.560391,0.374998,3.376912,8.935388,largeAF,color,5
2781,2,3.376912,stim,1,NaN,43.0,0.50,0.050,8.935388,0.374960,3.751871,9.310348,largeAF,color,5


In [103]:
duration = 1

df = all_logs[all_logs.run == '5']

stim_df = df[df.event_type =='stim']
switch_loc = np.diff(stim_df['color_balance'], prepend=0.5) != 0
switch_loc = stim_df[(switch_loc) & (stim_df.color_balance != 0.5)].index # drop values where color_balance is 0.5
responses = df.loc[df.response == 'space']

# check if there is a correct response within duration of stimulus following switch 
tp = [(abs(i - responses.onset) < duration).any() \
      for i in stim_df.loc[switch_loc].end] # true positives

fn = stim_df.loc[switch_loc][np.invert(tp)] # false negatives (missed switches)
fp = len(responses) - sum(tp) # false positives (responded with no switch)
tn = len(stim_df) - len(switch_loc) - len(fn) # true negative
accuracy = (sum(tp) + tn)/(sum(tp)+len(fn)+fp+tn)

print(f'Total stim: {len(stim_df)}\nTotal switches: {len(tp)}\nTP: {sum(tp)}\nTN : {tn}\nFP : {fp}\nFN : {len(fn)}\nAccuracy : {accuracy}')

Total stim: 897
Total switches: 176
TP: 68
TN : 613
FP : -28
FN : 108
Accuracy : 0.8948751642575559


In [21]:
df['stim_nr']= (df.trial_nr*2 + (df.phase -1)/2)

In [22]:
df.head(25)

,trial_nr,onset,event_type,phase,response,nr_frames,color_balance,fix_intensity,onset_abs,duration,end,end_abs,task,color,run,stim_nr
0,0,0.028607,stim,0,NaN,39.0,0.5,0.0,2.631448,0.358969,0.387576,2.990417,smallAF,color,4,-0.5
1,0,0.387576,stim,1,NaN,30.0,0.5,0.0,2.990417,0.374337,0.761913,3.364753,smallAF,color,4,0.0
2,0,0.761913,stim,2,NaN,44.0,0.5,0.0,3.364753,0.370416,1.132329,3.735169,smallAF,color,4,0.5
3,0,1.132329,stim,3,NaN,30.0,0.5,0.0,3.735169,0.371225,1.503553,4.106394,smallAF,color,4,1.0
4,1,1.503553,stim,0,NaN,45.0,0.5,0.0,4.106394,0.378696,1.882250,4.485090,smallAF,color,4,1.5
5,1,1.882250,stim,1,NaN,30.0,0.5,0.0,4.485090,0.371286,2.253535,4.856376,smallAF,color,4,2.0
6,1,2.253535,stim,2,NaN,45.0,0.5,0.0,4.856376,0.378654,2.632190,5.235030,smallAF,color,4,2.5
7,1,2.632190,stim,3,NaN,30.0,0.5,0.0,5.235030,0.371330,3.003520,5.606361,smallAF,color,4,3.0
8,2,3.003520,stim,0,NaN,45.0,0.5,0.0,5.606361,0.378589,3.382108,5.984949,smallAF,color,4,3.5
9,2,3.382108,stim,1,NaN,30.0,0.5,0.0,5.984949,0.371396,3.753504,6.356345,smallAF,color,4,4.0
